### Alumno : Rubén Álvarez Mosquera
---

# N Queens
---
---

#### Importamos librerías
---

In [13]:
# Library import 
###################################################

import os, sys, copy, time

#### Declaramos variables
---

In [14]:
# Declaracion variables
###################################################
version = 'v0.3'

limpia_pantalla = 'clear' if os.name == 'posix' else 'CLS'
celdas = 0
reinas = 0
estado_inicial = []
estado_actual = []
cerrados=[]
iteracion = 0
soluciones = []

#### Declaramos las funciones
---

##### Función : Reset:
- Esta función, realiza un reseteo de las variables usadas en el programa




In [15]:
def Reset():
    global cerrados, celdas, reinas, estado_actual,estado_inicial, iteracion, speed, soluciones #,tablero

    celdas = 0
    reinas = 0
    estado_inicial = []
    estado_actual = []
    iteracion = 0
    speed = 0
    cerrados=[]
    soluciones = []

##### Función : Limpia Pantalla:
- Esta función realizará una limpieza de la pantalla


In [16]:
def Limpia_pantalla():
    os.system(limpia_pantalla)

##### Función : Petición de variables:
- Esta función realizará una petición de las variable que necesita el programa, en este caso:
   - nº de celdas de ancho del tablero
   - nº de Reinas
   - Velocidad de muestreo


In [27]:
def Peticion_variables():
    global celdas, reinas, estado_inicial, estado_actual, speed

    Limpia_pantalla()
    print('\n====================')
    print('  N reinas   ',version)
    print('====================\n')

    print('Bienvenidos al juego de las N reinas \n')

    while celdas == 0 or not celdas.isnumeric():
        print('Tamaño del tablero, NxN celdas : (Recomendado no más de 5): ')
        celdas = input()
    celdas = int(celdas)
    while reinas == 0 or not reinas.isnumeric():
        print('Introduce el nº de reinas (Recomendado no más de 5): ')
        reinas = input()
    reinas = int(reinas)

    while speed == 0 or not speed.isnumeric():
        print('Introduce la velocidad (recomendado de 1 a 30+) : ', end='')
        speed = input()
    speed = float(speed)

    for x in range(celdas):
        estado_inicial.append([])
        for y in range(celdas):
            estado_inicial[x].append(0)

    estado_actual = copy.deepcopy(estado_inicial)

##### Función : Imprime Estado:
- Esta función imprime el resultado de cada uno de los movimientos que realiza el programa para conseguir la jugada adecuada



In [28]:
def Imprime_estado(state):
    print('\n====================')
    print('  N reinas   ',version)
    print('====================\n')
    print('Celdas : ',celdas,' x ',celdas)
    print('Reinas: ',reinas)
    print('\n====================\n')
    for x in range(len(state)):
        for y in range(len(state[x])):
            print(state[x][y],end=' ')
        print()
    print()
    print('Iteraciones : ',iteracion)
    print()
    print('Soluciones encontradas : ')
    if soluciones:
         for x in range(len(soluciones)):
            print(x+1,' : ',soluciones[x])
    time.sleep(1/speed)


##### Funciones de comprobación de estado:
- En estas funciones, el programa realiza una comprobación, para ver si las filas, columnas y diagonales(izquierda/derecha) del estado, cumplen las condiciones para ser un estado válido



In [29]:
def comprueba_filas(state):
    for x in range(len(state)):
        count = 0
        for y in range(len(state[x])):
            count += state[x][y]
        if count > 1: return False
    return True

def comprueba_columnas(state):
    for x in range(len(state)):
        count = 0
        for y in range(len(state[x])):
            count += state[y][x]
        if count > 1: return False
    return True

def comprueba_diagonales_der(state):    
    for x in range(len(state)):
        for y in range(len(state)):
            count=0
            for z in range(len(state)):
                try:
                    count += state[z+x][z+y]
                except:
                    pass
            if count > 1: return False
    return True

def inversa(state):
    inversa = copy.deepcopy(state)
    for x in range(len(state)):
        for y in range(len(state[x])):
            inversa[x][y] = state[x][-y-1]
    return inversa

def comprueba_diagonales_izq(state):
    if comprueba_diagonales_der(inversa(state)): return True
    else:                                        return False

def Comprueba_estado(state):
    if comprueba_filas(state) and comprueba_columnas(state) and comprueba_diagonales_der(state) and comprueba_diagonales_izq(state):
        return True
    else:
        return False

##### Función : Cuenta Reinas:
- Realiza un conteo de las reinas totales del estado, devolviendo un entero con el número de ellas

In [30]:
def Cuenta_reinas(state):
    count = 0
    for x in range(len(state)):
        for y in range(len(state)):
            count += state[x][y]
    return count

##### Función : Estados:
- Realiza una comprobación si existe una reina en cada una de las celdas, y si es así, pasa al la siguiente celda, y si está libre, coloca una reina y devuelve el estado.

In [31]:
def Estados(s):
    for x in range(len(s)):
        for y in range(len(s)):
            ns=copy.deepcopy(s)
            if ns[x][y] == 0:
                ns[x][y] = 1
                yield ns


##### Función :  Profundidad:
- En esta función realiza una comprobación del estado, devolviendo el estado actual, si se cumple que el número de reinas es el número objetivo, y si no, comprueba de cada uno de los estados siguientes posibles dados por la función "Compueba_estado()" si es un estado válido y no está en la lista de cerrados, agregándolo a la lista de cerrados en ese caso, y usando recursividad, para volver a realizar la misma comprobación.



In [32]:
def Profundidad(actual, path=[]):
    global iteracion
    if Cuenta_reinas(actual) == reinas:
        return [actual]
    else: 
        for ns in Estados(actual):
            iteracion += 1
            Limpia_pantalla()
            Imprime_estado(ns)
            if Comprueba_estado(ns) and ns not in cerrados:
                cerrados.append(ns)
                r=Profundidad(ns,cerrados)
                if r:
                    r.insert(0,actual)
                    return r
        return []

##### Función : Ejecución total:
- Esta función la usamos para ejecutar el grueso del programa, donde realizaremos:
   - Un reset de las variables
   - Petición de las variables
   - Intentaremos el código, evitando (cuando se acaben los índices de la búsqueda) que arroje un error de "IndexError", por haberllegado al final de los indices posibles, encontrando así el final del programa, y muestre la solución final o bien diciendo que no existe solución



In [33]:
def Ejecucion_total():
    global estado_actual,iteracion
    try:
        Reset()
        Peticion_variables()
        while True:
            solucion = Profundidad(estado_inicial)
            soluciones.append(solucion[-1])
            solucion = []
            print('Conseguido!!\n')
    except IndexError:
        print('Ha llegado al final.\n\nEl programa ha encontrado ',len(soluciones),'soluciones\n\nGracias por usar N-Reinas!!\n')


##### Programa principal:
---
---



En este caso, como se puede observar, únicamente limpiamos la pantalla y lanzamos la función principal "Ejecucion_total()"

##### RECUERDE:

- Esta función realizará una petición de las variable que necesita el programa, en este caso:
   - nº de celdas de ancho del tablero
   - nº de Reinas
   - Velocidad de muestreo

DISCLAIMER : Ciertas peticiones de variables mediante el método "input" aparecen erroneamente, así como otras salidas de pantalla que exceden el espacio que reserva el notebook para su muestreo.

Disculpen las molestias

---

In [ ]:
Ejecucion_total()